In [63]:
import cv2
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt
import pandas as pd

In [75]:
test_dir = "/Users/davidnull/phd/3D Module/data/force/testing/"
test_name = "10.3cm_test1"

csv_ext = ".csv"
mov_ext = ".MOV"

csv_file = test_dir + test_name + csv_ext
movie_file = test_dir + test_name + mov_ext

vid = cv2.VideoCapture(movie_file)

fps = round(vid.get(5))
print('Frames per second : ', fps,'FPS')

frame_count = vid.get(7)
print('Frame count : ', frame_count)

df = pd.read_csv(csv_file)
df = df.loc[df['stage'] == 6]
df = df.reset_index(drop=True)
exp_length = len(df)
df = df.assign(seconds=[0]*exp_length)
df = df.assign(sync=[0]*exp_length)
df = df.assign(force=[0]*exp_length)

df["seconds"] = df["timestamp"] / 1000000
start_time = df["seconds"].iloc[0]
df["seconds"] = df["seconds"] - start_time
print(df)

Frames per second :  30 FPS
Frame count :  2802.0
     stage     id   ping       mem    memExt  timestamp  init  drivers  \
0        6  15921  84723  0.621153  0.005118   56202747   255        0   
1        6  15921  79567  0.621153  0.005118   56252255   255        0   
2        6  15921  79567  0.621153  0.005118   56277318   255        0   
3        6  15921  79567  0.621153  0.005118   56303502   255        0   
4        6  15921  79567  0.621153  0.005118   56327228   255        0   
..     ...    ...    ...       ...       ...        ...   ...      ...   
520      6  15921  43254  0.626857  0.005118   71802204   255       78   
521      6  15921  43254  0.626857  0.005118   71828306   255       78   
522      6  15921  43254  0.627096  0.005118   71852302   255       78   
523      6  15921  43254  0.627096  0.005118   71879504   255       78   
524      6  15921  43254  0.627096  0.005118   71902505   255       78   

             p0         p1          p2          p3         ya

In [76]:
@interact(sync="", force="", frame_id=(0,frame_count,1))
def f(sync, force, frame_id):
    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = vid.read()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rotated = cv2.rotate(rgb_frame, cv2.ROTATE_90_CLOCKWISE)
    rotated = cv2.rotate(rotated, cv2.ROTATE_90_CLOCKWISE)
    cropped = rotated[200:600, 500:1000]
    plt.imshow(cropped)
    plt.show()
    if sync == "" or force == "":
        return
    sync = float(sync)
    force = float(force)
    if frame_id < sync:
        return 
    frames_since = frame_id - sync
    time_since = (1/fps) * frames_since
    place_id = (df['seconds']-time_since).abs().argsort()[0]
    df.at[place_id, "force"] = force
    df.at[place_id, "sync"] = sync
    return

interactive(children=(Text(value='', description='sync'), Text(value='', description='force'), FloatSlider(val…

In [77]:
df.to_csv(test_name + "_wforce" + csv_ext)